In [234]:
import pandas as pd
import numpy as np

df = pd.read_csv('data-cleaned-removed-empty.csv')
label=df["isNaturalCaused"]
df.drop(labels=["isNaturalCaused","fire_year", "true_cause", "general_cause_desc"], axis=1, inplace=True)
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

#Preprocessing:

categorical = [var for var in df.columns if df[var].dtype=='O']
numerical = [var for var in df.columns if var not in categorical]

In [235]:
def gini(isNaturalCaused, notNaturalCaused):
    total = isNaturalCaused + notNaturalCaused
    if total == 0:
        return 0
    return 1 - ((isNaturalCaused/total)**2 + (notNaturalCaused/total)**2)

def gini_split_categorical(data_frame, attribute, labels):
    gini_split = 0
    unique_vals = data_frame[attribute].unique()

    for val in unique_vals:
        isNaturalCaused = data_frame[(data_frame[attribute] == val) & (labels == 1)].shape[0]
        notNaturalCaused = data_frame[(data_frame[attribute] == val) & (labels == 0)].shape[0]

        gini_split += gini(isNaturalCaused, notNaturalCaused) * (isNaturalCaused + notNaturalCaused) / data_frame.shape[0]

    return gini_split

def gini_split_numerical(data_frame, attribute, labels):
    gini_split = 0
    unique_vals = data_frame[attribute].unique()
    
    min_gini_split = 1
    min_gini_split_point = 0

    for val in unique_vals:
        isNaturalCausedLeft = data_frame[(data_frame[attribute] <= val) & (labels == 1)].shape[0]
        notNaturalCausedLeft = data_frame[(data_frame[attribute] <= val) & (labels == 0)].shape[0]
        isNaturalCausedRight = data_frame[(data_frame[attribute] > val) & (labels == 1)].shape[0]
        notNaturalCausedRight = data_frame[(data_frame[attribute] > val) & (labels == 0)].shape[0]

        gini_split = gini(isNaturalCausedLeft, notNaturalCausedLeft) * (isNaturalCausedLeft + notNaturalCausedLeft) / data_frame.shape[0] + gini(isNaturalCausedRight, notNaturalCausedRight) * (isNaturalCausedRight + notNaturalCausedRight) / data_frame.shape[0]

        if gini_split < min_gini_split:
            min_gini_split = gini_split
            min_gini_split_point = val

    gini_split = min_gini_split

    return gini_split, min_gini_split_point

def get_split(data_frame, labels):
    min_gini = 1.0
    min_gini_attribute = ""
    min_gini_split_point = 0

    for attribute in data_frame.columns:
        if attribute in categorical:
            g = gini_split_categorical(data_frame, attribute, labels)
        elif attribute in numerical:
            g, split_point = gini_split_numerical(data_frame, attribute, labels)
            min_gini_split_point = split_point

        if g < min_gini:
            min_gini = g
            min_gini_attribute = attribute

    return min_gini_attribute, min_gini_split_point, min_gini

In [236]:
class node:
    def __init__(self, data_frame, labels):
        self.children = []
        self.data_frame = data_frame
        self.labels = labels

        self.split_attr, self.split_point, self.gini = get_split(self.data_frame, self.labels)

        print(data_frame)
        print(self.split_attr, self.split_point, self.gini)

        if self.gini == 0:
            self.pure = True
        elif data_frame.shape[0] != 0:
            self.pure = False
            if self.split_point:
                left_child_mask = data_frame[self.split_attr] <= self.split_point
                right_child_mask = data_frame[self.split_attr] > self.split_point
                print("left")
                self.children.append(node(data_frame[left_child_mask].drop(columns=[self.split_attr]), labels[left_child_mask]))
                print("right")
                self.children.append(node(data_frame[right_child_mask].drop(columns=[self.split_attr]), labels[right_child_mask]))
            else:
                unique_vals = data_frame[self.split_attr].unique()
                for val in unique_vals:
                    print("child", self.split_attr, val)
                    mask = data_frame[self.split_attr] == val
                    self.children.append(node(data_frame[mask].drop(columns=[self.split_attr]), labels[mask]))


class decision_tree:
    def __init__(self):
        pass

    def train(self, data, labels):
        self.root = node(data, labels)

    def predict(self, data):
        pass

In [237]:
tree = decision_tree()
tree.train(df, label)

# print(gini_split_numerical(df, "fire_spread_rate", label))

       assessment_hectares  current_size  fire_spread_rate fire_type  \
0                     0.01          0.10               0.0   Surface   
1                     0.20          0.20               0.0   Surface   
2                     0.50          0.50               0.0   Surface   
3                     0.01          0.01               0.0   Surface   
4                     0.10          0.10               0.1   Surface   
...                    ...           ...               ...       ...   
22474                 0.40          0.41               0.5   Surface   
22475                 0.10          0.10               0.0   Surface   
22476                 0.30          0.30               1.0    Ground   
22477                 0.01          0.01               0.0   Surface   
22478                 0.01          0.01               0.0   Surface   

      fire_position_on_slope weather_conditions_over_fire  temperature  \
0                       Flat                        Clear    

UFuncTypeError: ufunc 'greater_equal' did not contain a loop with signature matching types (<class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.StrDType'>) -> None